<a href="https://colab.research.google.com/github/ludoveltz/test_github_fev25/blob/main/Daily_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import userdata

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY is None:
        raise KeyError
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
except KeyError:
    print("⚠️ Veuillez configurer votre clé API dans les secrets Colab")

# Vérification de sécurité
if "GOOGLE_API_KEY" in os.environ:
    key = os.environ["GOOGLE_API_KEY"]
    print(f"Clé API configurée : {key[:4]}...{key[-4:]}")
else:
    print("❌ Clé API non configurée")



Clé API configurée : AIza...ER90


In [33]:
# Imports standards Python
from collections.abc import Iterable
from random import randint
from typing import Annotated, Literal

# Imports typing
from typing_extensions import TypedDict

# Imports LangGraph
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, InjectedState

# Imports LangChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.tools import tool
from langchain_core.messages.tool import ToolMessage

# Imports pour visualisation
from IPython.display import display
from graphviz import Digraph


In [ ]:
# 2. Définition de l'état
class OrderState(TypedDict):
    messages: Annotated[list, add_messages]
    order_items: list[str]
    finished: bool

# 3. Définition des constantes
BARISTABOT_SYSINT = SystemMessage(content="""Vous êtes BaristaBot, un système interactif de commande pour café.
Vous devez répondre exclusivement en français.

Vos responsabilités :
1. Prendre les commandes des clients à partir du menu
2. Répondre aux questions sur les produits et leur histoire
3. Gérer uniquement les discussions liées aux produits du menu
4. Structurer et confirmer les commandes avant leur finalisation

Fonctionnalités à utiliser :
- Utilisez get_menu pour consulter les produits disponibles
- Utilisez add_to_order pour ajouter des articles à la commande
- Utilisez clear_order pour réinitialiser une commande
- Utilisez get_order pour vérifier le contenu actuel de la commande
- Utilisez confirm_order pour valider la commande avec le client
- Utilisez place_order pour finaliser la commande

Instructions importantes :
- Vérifiez toujours que les boissons et les modifications demandées sont bien au menu
- Demandez des précisions si une commande n'est pas claire
- Confirmez toujours la commande avec le client avant de la finaliser
- Attendez la validation du client avant d'utiliser place_order
- Remerciez le client et prenez congé une fois la commande finalisée

Utilisez systématiquement l'outil get_menu pour vérifier la disponibilité des produits avant de les proposer ou de les ajouter à la commande.""")

WELCOME_MSG = "Bienvenue chez BaristaBot ! Tapez 'q' pour quitter. Que puis-je vous servir aujourd'hui ?"

# 4. Définition des outils
@tool
def get_menu() -> str:
    """Fournit le menu à jour."""
    return """
    MENU:
    Boissons au Café:
    Espresso
    Americano
    Cold Brew

    Boissons au Café avec Lait:
    Latte
    Cappuccino
    Cortado
    Macchiato
    Mocha
    Flat White

    Thés:
    English Breakfast Tea
    Green Tea
    Earl Grey

    Thés avec Lait:
    Chai Latte
    Matcha Latte
    London Fog

    Autres Boissons:
    Steamer
    Hot Chocolate

    Modificateurs:
    Options de lait: Entier, 2%, Avoine, Amande, 2% Sans Lactose; Option par défaut: entier
    Shots d'espresso: Simple, Double, Triple, Quadruple; défaut: Double
    Caféine: Décaféiné, Normal; défaut: Normal
    Chaud-Glacé: Chaud, Glacé; Défaut: Chaud
    Édulcorants: sirop vanille, sirop noisette, sauce caramel, sauce chocolat, sirop vanille sans sucre
    Demandes spéciales: toute modification raisonnable n'impliquant pas d'articles hors menu

    Note: Le lait de soja est en rupture de stock aujourd'hui.
    """

tool
def add_to_order(drink: str, modifiers: Iterable[str]) -> str:
    """Ajoute la boisson spécifiée à la commande du client, avec ses modificateurs.
    Returns:
        La commande mise à jour.
    """
    pass

@tool
def confirm_order() -> str:
    """Demande au client si la commande est correcte.
    Returns:
        La réponse du client.
    """
    pass

@tool
def get_order() -> str:
    """Retourne la commande en cours.
    Returns:
        La liste des articles commandés, un par ligne.
    """
    pass

@tool
def clear_order() -> None:
    """Supprime tous les articles de la commande."""
    pass

@tool
def place_order() -> int:
    """Envoie la commande au barista.
    Returns:
        Temps estimé en minutes avant que la commande soit prête.
    """
    pass

def order_node(state: OrderState) -> OrderState:
    """Nœud de gestion des commandes."""
    if not (msgs := state.get("messages", [])):
        return state

    tool_msg = msgs[-1]
    order = state.get("order_items", [])
    outbound_msgs = []
    order_placed = False

    for tool_call in tool_msg.tool_calls:
        if tool_call["name"] == "add_to_order":
            modifiers = tool_call["args"].get("modifiers", [])
            modifier_str = ", ".join(modifiers) if modifiers else "sans modification"

            order.append(f'{tool_call["args"]["drink"]} ({modifier_str})')
            response = "\n".join(order)

        elif tool_call["name"] == "confirm_order":
            print("Votre commande :")
            if not order:
                print("  (aucun article)")

            for drink in order:
                print(f"  {drink}")

            response = input("Est-ce correct ? ")

        elif tool_call["name"] == "get_order":
            response = "\n".join(order) if order else "(aucune commande)"

        elif tool_call["name"] == "clear_order":
            order.clear()
            response = None

        elif tool_call["name"] == "place_order":
            order_text = "\n".join(order)
            print("Envoi de la commande en préparation !")
            print(order_text)

            order_placed = True
            response = randint(1, 5)  # Temps estimé en minutes

        else:
            raise NotImplementedError(f'Outil inconnu : {tool_call["name"]}')

        outbound_msgs.append(
            ToolMessage(
                content=str(response),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )

    return {"messages": outbound_msgs, "order_items": order, "finished": order_placed}

# 5. Configuration des outils et du graphe
# Séparation des outils automatiques et de commande
auto_tools = [get_menu]
tool_node = ToolNode(auto_tools)

# Outils de commande gérés par order_node
order_tools = [add_to_order, confirm_order, get_order, clear_order, place_order]

# Configuration du LLM avec tous les outils
all_tools = auto_tools + order_tools
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")
llm_with_tools = llm.bind_tools(all_tools)

# 6. Définition des fonctions de nœud
def chatbot_with_tools(state: OrderState) -> OrderState:
    """Le chatbot avec outils."""
    defaults = {"order_items": [], "finished": False}

    if state["messages"]:
        new_output = llm_with_tools.invoke([BARISTABOT_SYSINT] + state["messages"])
    else:
        new_output = AIMessage(content=WELCOME_MSG)

    return defaults | state | {"messages": [new_output]}

def human_node(state: OrderState) -> OrderState:
    """Nœud d'interaction humaine."""
    last_msg = state["messages"][-1]
    print("BaristaBot:", last_msg.content)

    user_input = input("Vous: ")
    if user_input.lower() in {"q", "quit", "exit", "goodbye", "au revoir"}:
        state["finished"] = True

    return state | {"messages": [HumanMessage(content=user_input)]}

# 7. Fonctions de routage
def maybe_route_to_tools(state: OrderState) -> str:
    """Route entre les différents nœuds selon le type de message."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"Aucun message trouvé dans l'état : {state}")

    msg = msgs[-1]

    if state.get("finished", False):
        return END
    elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        if any(tool["name"] in tool_node.tools_by_name.keys() for tool in msg.tool_calls):
            return "tools"
        else:
            return "ordering"
    else:
        return "human"

def maybe_exit_human_node(state: OrderState) -> Literal["chatbot", "__end__"]:
    """Route vers le chatbot sauf si l'utilisateur sort."""
    if state.get("finished", False):
        return END
    return "chatbot"

# 8. Configuration du graphe
graph_builder = StateGraph(OrderState)

# Ajout des nœuds
graph_builder.add_node("chatbot", chatbot_with_tools)
graph_builder.add_node("human", human_node)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("ordering", order_node)

# Configuration des transitions
graph_builder.set_entry_point("chatbot")

# Chatbot -> {ordering, tools, human, END}
graph_builder.add_conditional_edges(
    "chatbot",
    maybe_route_to_tools,
    {
        "human": "human",
        "tools": "tools",
        "ordering": "ordering",
        "end": END
    }
)

# Human -> {chatbot, END}
graph_builder.add_conditional_edges(
    "human",
    maybe_exit_human_node,
    {
        "chatbot": "chatbot",
        "end": END
    }
)

# Les outils retournent toujours vers le chat
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("ordering", "chatbot")

# Point d'entrée explicite
graph_builder.add_edge(START, "chatbot")

# Compilation du graphe final
graph_with_order_tools = graph_builder.compile()

# Visualisation du graphe
from IPython.display import Image
Image(graph_with_order_tools.get_graph().draw_mermaid_png())


# 9. Configuration et lancement
from pprint import pprint

# Configuration avec limite de récursion augmentée
config = {"recursion_limit": 100}

# 10. Lancement du chat
if __name__ == "__main__":
    initial_state = {
        "messages": [],
        "order_items": [],
        "finished": False
    }

    # Lancement avec configuration
    state = graph_with_order_tools.invoke(initial_state, config)

    # Affichage de l'état final
    pprint(state)


BaristaBot: Bienvenue chez BaristaBot ! Tapez 'q' pour quitter. Que puis-je vous servir aujourd'hui ?
